<a href="https://colab.research.google.com/github/nvarasp/FinalProject_IRWA/blob/main/notebook/RQ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RQ2 - Output diversification

### 0. Imports and configuration

In [1]:
%%time
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CPU times: user 1.16 ms, sys: 0 ns, total: 1.16 ms
Wall time: 1.37 ms


In [2]:
# Insert the directory
import sys
sys.path.insert(0,"/content/drive/My Drive/IRWA/FinalProject/notebook/")

In [3]:
!pip install config

In [4]:
from functions_v3 import *
from gensim.models import Word2Vec
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from sklearn.cluster import KMeans
import seaborn as sns
from wordcloud import WordCloud, ImageColorGenerator
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import operator
from scipy.stats import spearmanr
import ast
import csv
STOPWORDS = set(stopwords.words("english"))
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
IMPORTS ="/content/drive/My Drive/IRWA/FinalProject/data/" 
CLUSTERS = '/content/drive/My Drive/IRWA/FinalProject/data/clustered_data.csv'

Read the processed data and generate index from it. Retrieve the term frequency, document frequency, and inverse document frequency.

In [5]:
df_proc = pd.read_csv(IMPORTS+'df_to_cluster.csv')
df_proc['Tweet_proc'] = [list(ast.literal_eval(x)) for x in df_proc['Tweet_proc']]
df_proc.head()

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,Tweet_proc
0,@JoeBiden R/T IF YOU AGREE!!\n#JoeBidenKamalaH...,SarxClothing_,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'JoeBidenKamalaHarris2020', 'indices...",0,0,https://twitter.com/twitter/statuses/133592651...,"[joebiden, rt, agree, joebidenkamalaharris2020..."
1,RT @dblackanese: It’s a very big deal. \n#Wear...,nightnum9,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'WearAMask', 'indices': [40, 50]}]",0,4400,https://twitter.com/twitter/statuses/133592651...,"[dblackanese, big, deal, wearamask, httpstcor9..."
2,RT @Amber02150: Wishing everyone a nice health...,jackwen5566,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'StaySafe', 'indices': [55, 64]}, {'...",0,400,https://twitter.com/twitter/statuses/133592651...,"[amber02150, wishing, everyone, nice, healthy,..."
3,RT @dblackanese: It’s a very big deal. \n#Wear...,jmcmichael2017,Mon Dec 07 12:37:56 +0000 2020,"[{'text': 'WearAMask', 'indices': [40, 50]}]",0,4400,https://twitter.com/twitter/statuses/133592651...,"[dblackanese, big, deal, wearamask, httpstcor9..."
4,@JoeBiden R/T IF YOU AGREE!!\n#JoeBidenKamalaH...,SarxClothing_,Mon Dec 07 12:37:57 +0000 2020,"[{'text': 'JoeBidenKamalaHarris2020', 'indices...",0,0,https://twitter.com/twitter/statuses/133592652...,"[joebiden, rt, agree, joebidenkamalaharris2020..."


In [6]:
start_time = time.time()
numDocuments = len(df_proc["Tweet_proc"])
index, tf, df, idf= create_tf_idf_index(df_proc,"Tweet_proc", numDocuments)

In [7]:
df_proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16644 entries, 0 to 16643
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tweet       16644 non-null  object
 1   Username    16644 non-null  object
 2   Date        16644 non-null  object
 3   Hashtags    16644 non-null  object
 4   Likes       16644 non-null  int64 
 5   Retweets    16644 non-null  int64 
 6   Url         16644 non-null  object
 7   Tweet_proc  16644 non-null  object
dtypes: int64(2), object(6)
memory usage: 1.0+ MB


### 1. Clustered data

In the previous research question, we decided to assign 4 clusters to our data, using the elbow criterion.

Read the processed data with the cluster label assigned.

In [8]:
df_clusters = pd.read_csv(CLUSTERS)
num_clusters = len(df_clusters.Cluster.unique())
df_clusters.head()

,Unnamed: 0,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,Tweet_proc,Cluster
0,0,@JoeBiden R/T IF YOU AGREE!!\n#JoeBidenKamalaH...,SarxClothing_,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'JoeBidenKamalaHarris2020', 'indices...",0,0,https://twitter.com/twitter/statuses/133592651...,"['joebiden', 'rt', 'agree', 'joebidenkamalahar...",1
1,1,RT @dblackanese: It’s a very big deal. \n#Wear...,nightnum9,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'WearAMask', 'indices': [40, 50]}]",0,4400,https://twitter.com/twitter/statuses/133592651...,"['dblackanese', 'big', 'deal', 'wearamask', 'h...",0
2,2,RT @Amber02150: Wishing everyone a nice health...,jackwen5566,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'StaySafe', 'indices': [55, 64]}, {'...",0,400,https://twitter.com/twitter/statuses/133592651...,"['amber02150', 'wishing', 'everyone', 'nice', ...",0
3,3,RT @dblackanese: It’s a very big deal. \n#Wear...,jmcmichael2017,Mon Dec 07 12:37:56 +0000 2020,"[{'text': 'WearAMask', 'indices': [40, 50]}]",0,4400,https://twitter.com/twitter/statuses/133592651...,"['dblackanese', 'big', 'deal', 'wearamask', 'h...",0
4,4,@JoeBiden R/T IF YOU AGREE!!\n#JoeBidenKamalaH...,SarxClothing_,Mon Dec 07 12:37:57 +0000 2020,"[{'text': 'JoeBidenKamalaHarris2020', 'indices...",0,0,https://twitter.com/twitter/statuses/133592652...,"['joebiden', 'rt', 'agree', 'joebidenkamalahar...",1


Now, we count the number of tweets per label.

In [9]:
Counter(df_clusters.Cluster)

Counter({0: 1135, 1: 12233, 2: 1698, 3: 1578})

We can see that most of the tweets belong to the first cluster. This bias in the data will affect ranking.

### 2. Diversity score

In this section, we assign to each of the rankings by query a diversity score, that indicates how much diversity in the labels there is. First, we retrieve the initial rankings for a list of queries.

In [10]:
queries = ["new coronavirus cases", "covid vaccine", "kevincorke", "country with more cases", "husband got", "hospital workers", "global pandemic", "tomfitton", "covid deaths","stayhome in christmas", "test results", "#wearamask"]

In [11]:
with open('/content/drive/My Drive/IRWA/FinalProject/other-outputs/output_RQ2_a.tsv', 'wt') as out_file:
  tsv_writer = csv.writer(out_file, delimiter='\t')
  normal_rankings = []
  for i in range (0,len(queries)):
    print("Query: ", queries[i])
    #print("Insert your query:\n")
    #query = input()
    ranked_docs = search_tf_idf(queries[i], index, tf, idf)    
    top = 20
    ranking = []

    tsv_writer.writerow(["QUERY: ", queries[i]])
    tsv_writer.writerow(["Ranking", "Tweet", "Text", "Username", "Date", "URL", "Retweets", "Likes", "Hashtags", "Cluster"])
    tsv_writer.writerow([" "])

    if(top<=len(ranked_docs)):
      print("\n======================\nTop {} results out of {} for the seached query:\n".format(top, len(ranked_docs)))
      n=1
      for d_id in ranked_docs[:top] :
        tsv_writer.writerow([n, d_id, df_proc.iloc[d_id][0], df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6], df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3], df_clusters.Cluster.iloc[d_id]])
        print("{} tweet: {} \n".format(n, d_id))
        print("Text: {}.\n".format(df_proc.iloc[d_id][0]))
        print("Username: {}. Date: {}. URL: {}\n".format(df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6]))
        print("Retweets: {}. Likes: {}. Hashtags: {}\n".format(df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3]))
        print("Cluster: {}.\n".format(df_clusters.Cluster.iloc[d_id]))
        print("\n")
        n+=1
        ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])
      normal_rankings.append(ranking)
    else:
      print("\n======================\nTop {} results out of {} for the seached query:\n".format(len(ranked_docs), len(ranked_docs)))
      n=1
      for d_id in ranked_docs[:len(ranked_docs)]:
        tsv_writer.writerow([n, d_id, df_proc.iloc[d_id][0], df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6], df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3], df_clusters.Cluster.iloc[d_id]])
        print("{} tweet: {} \n".format(n, d_id))
        print("Text: {}.\n".format(df_proc.iloc[d_id][0]))
        print("Username: {}. Date: {}. URL: {}\n".format(df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6]))
        print("Retweets: {}. Likes: {}. Hashtags: {}\n".format(df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3]))
        print("Cluster: {}.\n".format(df_clusters.Cluster.iloc[d_id]))
        print("\n")
        n+=1
        ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])
      normal_rankings.append(ranking)

Query:  new coronavirus cases

Top 20 results out of 260 for the seached query:

1 tweet: 14359 

Text: #CORONAVIRUS UPDATE: The state reported 3,817 new cases of the #COVID19 this morning — this is the second-largest daily increase of new cases since the pandemic began. The largest increase was recorded yesterday at 3,880 new cases. https://t.co/ATAq5P7ry1 #Virginia.

Username: 8NEWS. Date: Mon Dec 07 15:53:05 +0000 2020. URL: https://twitter.com/twitter/statuses/1335975628127035394

Retweets: 1. Likes: 3. Hashtags: [{'text': 'CORONAVIRUS', 'indices': [0, 12]}, {'text': 'COVID19', 'indices': [63, 71]}, {'text': 'Virginia', 'indices': [256, 265]}]

Cluster: 1.



2 tweet: 4406 

Text: RT @BangkokPostNews: #ChiangRai reported 6 more #coronavirus #COVID19 cases from Thais returning from Tachilek. The new cases have not been….

Username: alexchoe21. Date: Mon Dec 07 13:54:06 +0000 2020. URL: https://twitter.com/twitter/statuses/1335945683447136263

Retweets: 11. Likes: 0. Hashtags: [{'tex

Then, we compute the diversity score. This metric has a high value if the ranking contains tweets from a balanced set of clusters. On the other hand, if all tweets in the ranking belong to the same class, then the metric will be low. To do so, we count the number of cluster ocurrences in the ranking and construct a num_clusters-dimensional vector. Also, we calculate the ideal counts for the ranking. For example, if we have 4 clusters and 20 tweets in the ranking, the ideal vector is [5 5 5 5]. To compute the final score, we subtract to 1 the cosine distance between the two vectors: the real count vector and the ideal count vector. In this way, vectors that are close to the ideal ranking will have a diversity score of 1.

In [12]:
def get_ideal_counts(size, num_clusters):
  ideal_count = [size // num_clusters + (1 if x < size % num_clusters else 0)  for x in range(num_clusters)]
  return ideal_count

In [13]:
def get_diversity_score(ranking, num_clusters):
  #divide number of results into cluster buckets
  ideal_count = get_ideal_counts(len(ranking), num_clusters)
  clusters = []
  for element in ranking:
     clusters.append(element[1])
  actual_count = Counter(clusters)
  for i in range(num_clusters):
    if i not in actual_count:
      actual_count[i] = 0

  a = np.array(ideal_count)
  b = np.array(list(actual_count.values()))
  score = 1 - spatial.distance.cosine(a, b)
  #print("The diversity score is " + str(score))
  return score

In [14]:
for ranking in normal_rankings:
  get_diversity_score(ranking, num_clusters)


For example, we can show a toy example of our metric. If the cluster counts are near to the ideal (vector $c$), we get a high diversity score (close to 1).

In [15]:
c = np.array([4, 6, 7, 3])
ideal = np.array([5,5,5,5])
result = 1 - spatial.distance.cosine(c, ideal)
result

0.9534625892455922

On the other hand, if the counts are very unbalance, the metric will give a lower result.

In [16]:
d = np.array([19, 1, 0, 0])
ideal = np.array([5,5,5,5])
result = 1 - spatial.distance.cosine(d, ideal)
result

0.5255883312276367

### 3. Re-ranking

In [17]:
threshold = 0.7

In [18]:
def combine_rankings(test_list1, test_list2):
  size_1 = len(test_list1) 
  size_2 = len(test_list2) 
    
  res = [] 
  i, j = 0, 0
    
  while i < size_1 and j < size_2: 
      if test_list1[i] < test_list2[j]: 
        res.append(test_list1[i]) 
        i += 1
    
      else: 
        res.append(test_list2[j]) 
        j += 1
    
  res = res + test_list1[i:] + test_list2[j:] 
  return res

In [19]:
def rerank(ranking, num_clusters, max_ranking, top):
  relevant_i = min(len(ranking), max_ranking)#filter out nonrelevant documents
  print("The maximum rank to consider an element relevant is: " + str(relevant_i))
  ranking = ranking[:relevant_i] 
  column_names = ["tweet_id", "cluster"]
  df_ranking = pd.DataFrame(data = ranking, columns = column_names)
  cluster_counts = Counter(df_ranking.cluster)
  print(cluster_counts)

  if (top<=len(ranking)):
    print("There are relevant results from %d clusters" %(len(cluster_counts.keys())))
    ranking_new = []
    ideal_count = get_ideal_counts(top, len(cluster_counts.keys()))    
    print("ideal count: "+ str(ideal_count))
    i = 0
    for cluster in cluster_counts.keys():
      df_cluster = df_ranking[df_ranking.cluster==cluster]
      if ideal_count[i]<=cluster_counts[cluster]:
        ranking_new = combine_rankings(ranking_new, list(df_cluster.index)[:ideal_count[i]])
      else:
        ranking_new = combine_rankings(ranking_new, list(df_cluster.index)[:cluster_counts[cluster]])
        diff = ideal_count[i] - cluster_counts[cluster]
        ideal_count[(i+1)%num_clusters] +=diff
      i+=1
  else:
    print("There are relevant results from %d clusters" %(len(cluster_counts.keys())))
    ranking_new = []
    ideal_count = get_ideal_counts(len(ranking), len(cluster_counts.keys()))
    i = 0
    for cluster in cluster_counts.keys():
      df_cluster = df_ranking[df_ranking.cluster==cluster]
      if ideal_count[i]<=cluster_counts[cluster]:
        ranking_new = combine_rankings(ranking_new, list(df_cluster.index)[:ideal_count[i]])
      else:
        ranking_new = combine_rankings(ranking_new, list(df_cluster.index)[:cluster_counts[cluster]])
        diff = ideal_count[i] - cluster_counts[cluster]
        ideal_count[(i+1)%num_clusters] +=diff
      i+=1

  df_ranking_new = df_ranking.iloc[ranking_new]
  return list(df_ranking_new.tweet_id)

In [20]:
with open('/content/drive/My Drive/IRWA/FinalProject/other-outputs/output_RQ2_b.tsv', 'wt') as out_file:
  tsv_writer = csv.writer(out_file, delimiter='\t')
  div_rankings = []
  for i in range (0,len(queries)):
    #print("Insert your query:\n")
    #query = input()
    print("Query: ", queries[i])
    tsv_writer.writerow(["QUERY: ", queries[i]])
    tsv_writer.writerow(["Ranking", "Tweet", "Text", "Username", "Date", "URL", "Retweets", "Likes", "Hashtags", "Cluster"])
    tsv_writer.writerow([" "])

    top = 20
    max_ranking = 20000
    diversity_score = 0
    ranking = []
    ranked_docs = search_tf_idf(queries[i], index, tf, idf)
    print("\n======================\n\nLenght of initial ranking: " + str(len(ranked_docs)))

    if(top<=len(ranked_docs)): 
      n=1
      for d_id in ranked_docs[:top] :
        n+=1
        ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])
        first_diversity_score = get_diversity_score(ranking[:top], num_clusters)
    else: #if there are less documents thank expected
      n=1
      for d_id in ranked_docs[:len(ranked_docs)]:
        n+=1
        ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])
        first_diversity_score = get_diversity_score(ranking[:len(ranked_docs)], num_clusters)
    div_ranking = ranking

    #check if there is ranking diversity
    if first_diversity_score < threshold: 
      global_ranking = []
      print("The results do not have not enough diversity.")
      for d_id in ranked_docs: #get full ranking
        global_ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])  
          
      new_docs = rerank(global_ranking, num_clusters, max_ranking, top)
      print("\nTop {} results out of {} for the seached query:\n".format(len(new_docs), len(new_docs)))
      n=1
      new_ranking = []
      for d_id in new_docs:
        tsv_writer.writerow([n, d_id, df_proc.iloc[d_id][0], df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6], df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3], df_clusters.Cluster.iloc[d_id]])
        print("{} tweet: {} \n".format(n, d_id))
        print("Text: {}.\n".format(df_proc.iloc[d_id][0]))
        print("Username: {}. Date: {}. URL: {}\n".format(df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6]))
        print("Retweets: {}. Likes: {}. Hashtags: {}\n".format(df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3]))
        print("Cluster: {}.\n".format(df_clusters.Cluster.iloc[d_id]))
        print("\n")
        new_ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])
        n+=1
      diversity_score = get_diversity_score(new_ranking, num_clusters)
      print("The first diversity score was: " + str(first_diversity_score) + "\n")
      print("New diversity score: " + str(diversity_score))
      print()
      div_ranking = new_ranking
    else:
      print("The raking is diverse.")

      if(top<=len(ranked_docs)):
        print("\nTop {} results out of {} for the seached query:\n".format(top, len(ranked_docs)))
        n=1
        for d_id in ranked_docs[:top] :
          tsv_writer.writerow([n, d_id, df_proc.iloc[d_id][0], df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6], df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3], df_clusters.Cluster.iloc[d_id]])
          print("{} tweet: {} \n".format(n, d_id))
          print("Text: {}.\n".format(df_proc.iloc[d_id][0]))
          print("Username: {}. Date: {}. URL: {}\n".format(df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6]))
          print("Retweets: {}. Likes: {}. Hashtags: {}\n".format(df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3]))
          print("Cluster: {}.\n".format(df_clusters.Cluster.iloc[d_id]))
          print("\n")
          n+=1
          
      else:
        print("Top {} results out of {} for the seached query:\n".format(len(ranked_docs), len(ranked_docs)))
        n=1
        for d_id in ranked_docs[:len(ranked_docs)]:
          tsv_writer.writerow([n, d_id, df_proc.iloc[d_id][0], df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6], df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3], df_clusters.Cluster.iloc[d_id]])
          print("{} tweet: {} \n".format(n, d_id))
          print("Text: {}.\n".format(df_proc.iloc[d_id][0]))
          print("Username: {}. Date: {}. URL: {}\n".format(df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6]))
          print("Retweets: {}. Likes: {}. Hashtags: {}\n".format(df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3]))
          print("Cluster: {}.\n".format(df_clusters.Cluster.iloc[d_id]))
          print("\n")
          n+=1
      print("The diversity score is: " + str(first_diversity_score) + "\n")
      print()
    div_rankings.append(div_ranking)
      
  


Query:  new coronavirus cases


Lenght of initial ranking: 260
The results do not have not enough diversity.
The maximum rank to consider an element relevant is: 260
Counter({1: 260})
There are relevant results from 1 clusters
ideal count: [20]

Top 20 results out of 20 for the seached query:

1 tweet: 14359 

Text: #CORONAVIRUS UPDATE: The state reported 3,817 new cases of the #COVID19 this morning — this is the second-largest daily increase of new cases since the pandemic began. The largest increase was recorded yesterday at 3,880 new cases. https://t.co/ATAq5P7ry1 #Virginia.

Username: 8NEWS. Date: Mon Dec 07 15:53:05 +0000 2020. URL: https://twitter.com/twitter/statuses/1335975628127035394

Retweets: 1. Likes: 3. Hashtags: [{'text': 'CORONAVIRUS', 'indices': [0, 12]}, {'text': 'COVID19', 'indices': [63, 71]}, {'text': 'Virginia', 'indices': [256, 265]}]

Cluster: 1.



2 tweet: 4406 

Text: RT @BangkokPostNews: #ChiangRai reported 6 more #coronavirus #COVID19 cases from Thais retur

In [21]:
normal_rankings

[[[14359, 1],
  [4406, 1],
  [4983, 1],
  [4749, 1],
  [4685, 1],
  [4679, 1],
  [4072, 1],
  [3280, 1],
  [3089, 1],
  [14396, 1],
  [977, 1],
  [972, 1],
  [12062, 1],
  [636, 1],
  [16521, 1],
  [5427, 1],
  [192, 1],
  [11623, 1],
  [1862, 1],
  [12659, 1]],
 [[9556, 1],
  [9515, 1],
  [9518, 1],
  [15983, 1],
  [14059, 1],
  [7209, 1],
  [617, 1],
  [15217, 1],
  [1192, 1],
  [1868, 1],
  [1124, 1],
  [10567, 1],
  [10389, 1],
  [11518, 1],
  [3165, 1],
  [3141, 1],
  [3080, 1],
  [1369, 1],
  [858, 1],
  [811, 1]],
 [[9159, 1],
  [15792, 1],
  [16567, 3],
  [16565, 3],
  [16548, 3],
  [16536, 3],
  [16510, 3],
  [16504, 3],
  [16489, 3],
  [16472, 3],
  [16467, 3],
  [16461, 3],
  [16451, 3],
  [16420, 3],
  [16404, 3],
  [16383, 3],
  [16371, 3],
  [16370, 3],
  [16369, 3],
  [16317, 3]],
 [[15272, 1],
  [16211, 1],
  [4201, 1],
  [2033, 1],
  [326, 1],
  [2145, 1],
  [12794, 1],
  [14566, 1],
  [14193, 1],
  [12680, 1],
  [2036, 1],
  [13010, 1],
  [16010, 1],
  [15568, 1],
  [

## 4. Report


### 4.1 Ranking Difference

We computed the Spearmans correlation coefficient to compare the two rankings: https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient

In [22]:
ranking_1 = []
ranking_2 = []
for rank in normal_rankings:  
  rank = [item[0] for item in rank]
  ranking_1.append(rank)
for rank in div_rankings:  
  rank = [item[0] for item in rank]
  ranking_2.append(rank)


In [23]:
for q, rank1,rank2 in zip(queries, ranking_1, ranking_2):
  print("Rankings for query: ", q)
  print("TF-IDF ranking: " + str(rank1))
  print("Diverse ranking: "+ str(rank2))
  coef, p = spearmanr(rank1, rank2)
  print('Spearmans correlation coefficient: %.3f' % coef)
  # interpret the significance
  alpha = 0.05
  if p > alpha:
    print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
  else:
    print('Samples are correlated (reject H0) p=%.3f' % p)
  print()

Rankings for query:  new coronavirus cases
TF-IDF ranking: [14359, 4406, 4983, 4749, 4685, 4679, 4072, 3280, 3089, 14396, 977, 972, 12062, 636, 16521, 5427, 192, 11623, 1862, 12659]
Diverse ranking: [14359, 4406, 4983, 4749, 4685, 4679, 4072, 3280, 3089, 14396, 977, 972, 12062, 636, 16521, 5427, 192, 11623, 1862, 12659]
Spearmans correlation coefficient: 1.000
Samples are correlated (reject H0) p=0.000

Rankings for query:  covid vaccine
TF-IDF ranking: [9556, 9515, 9518, 15983, 14059, 7209, 617, 15217, 1192, 1868, 1124, 10567, 10389, 11518, 3165, 3141, 3080, 1369, 858, 811]
Diverse ranking: [9556, 9515, 9518, 15983, 14059, 7209, 617, 15217, 1192, 1868, 1124, 10567, 10389, 11518, 3165, 3141, 3080, 1369, 858, 811]
Spearmans correlation coefficient: 1.000
Samples are correlated (reject H0) p=0.000

Rankings for query:  kevincorke
TF-IDF ranking: [9159, 15792, 16567, 16565, 16548, 16536, 16510, 16504, 16489, 16472, 16467, 16461, 16451, 16420, 16404, 16383, 16371, 16370, 16369, 16317]
Dive

### 4.2 Coverage

In [24]:
clusters_1 = []
clusters_2 = []
for rank in normal_rankings:  
  rank = [item[1] for item in rank]
  clusters_1.append(rank)
for rank in div_rankings:  
  rank = [item[1] for item in rank]
  clusters_2.append(rank)

In [25]:
def return_coverage(clusters_1,clusters_2,queries):
  coverage_1, coverage_2 = [], []
  for q, c1, c2 in zip(queries, clusters_1, clusters_2):
    print("Clusters for query: ", q)
    print("Coverage TF-IDF ranking: " + str(len(set(c1))))
    coverage_1.append(len(set(c1)))
    print("Coverage diversity ranking: " + str(len(set(c2))))
    coverage_2.append(len(set(c2)))
    print()
  return coverage_1, coverage_2

In [26]:
cov_1, cov_2 = return_coverage(clusters_1, clusters_2, queries)

Clusters for query:  new coronavirus cases
Coverage TF-IDF ranking: 1
Coverage diversity ranking: 1

Clusters for query:  covid vaccine
Coverage TF-IDF ranking: 1
Coverage diversity ranking: 1

Clusters for query:  kevincorke
Coverage TF-IDF ranking: 2
Coverage diversity ranking: 2

Clusters for query:  country with more cases
Coverage TF-IDF ranking: 1
Coverage diversity ranking: 1

Clusters for query:  husband got
Coverage TF-IDF ranking: 2
Coverage diversity ranking: 2

Clusters for query:  hospital workers
Coverage TF-IDF ranking: 1
Coverage diversity ranking: 1

Clusters for query:  global pandemic
Coverage TF-IDF ranking: 1
Coverage diversity ranking: 1

Clusters for query:  tomfitton
Coverage TF-IDF ranking: 1
Coverage diversity ranking: 2

Clusters for query:  covid deaths
Coverage TF-IDF ranking: 1
Coverage diversity ranking: 1

Clusters for query:  stayhome in christmas
Coverage TF-IDF ranking: 1
Coverage diversity ranking: 1

Clusters for query:  test results
Coverage TF-IDF